In [1]:
import math
import time
import pandas as pd
import pickle
import os.path as path
import numpy as np
#import matplotlib.pyplot as plt

#from sklearn.metrics import accuracy_score
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
import torch


Using TensorFlow backend.


In [2]:
data_path='C:/Users/YU007/CE7454_2018/project/quick_draw_output' # change path here

with open(path.join(data_path,'train_X'),'rb') as f:
    train_X=pickle.load(f)

with open(path.join(data_path,'train_Y'),'rb') as f:
    train_label=pickle.load(f)
    
with open(path.join(data_path,'test_X'),'rb') as f:
    test_X=pickle.load(f)

with open(path.join(data_path,'test_Y'),'rb') as f:
    test_label=pickle.load(f)

In [3]:
def convert_to_zeropad(data):
    stroke_no = np.zeros(len(data))
    point_no = []

    for i in range(len(data)): # number of pictures, ith picture
        stroke_no[i] = len(data[i])
        for j in range(len(data[i])): # number of strokes for each picture, jth stroke
            point_no.append(len(data[i][j][0]))

    stroke_no_max = int(max(stroke_no))
    point_no_max = int(max(point_no))

    print ('max stroke number in an image =',stroke_no_max,'\n max point number in a stroke =', point_no_max)
    print ('image number =', len(data))

   
    Xdata = np.zeros((len(data),stroke_no_max,2,point_no_max))

    for i in range(len(data)):  
        for j in range(len(data[i])):        
            Xdata[i][j][0][:len(data[i][j][0])]=data[i][j][0][:]
            Xdata[i][j][1][:len(data[i][j][0])]=data[i][j][1][:]

    Xdata_tensor = torch.Tensor(Xdata)
    Xdata_tensor = Xdata_tensor.permute(0, 1, 3, 2)
    Xdata_tensor = Xdata_tensor.reshape(len(data), stroke_no_max*point_no_max, 2)
    return np.array(Xdata_tensor)

In [4]:
train_data = convert_to_zeropad(train_X[:500])
print (train_data[0])

#trainY_Onehot = np.zeros((train_label.shape[0],5))
#trainY_Onehot[np.arange(train_label.shape[0]),train_label.astype(int)] = 1 


# define universe of possible input values

# one hot encode
onehot = []
for i in range(500):
    if train_label[i]==0:
        onehot.append([1,0,0,0,0])
    elif train_label[i]==1:
        onehot.append([0,1,0,0,0])
    elif train_label[i]==2:
        onehot.append([0,0,1,0,0])
    elif train_label[i]==3:
        onehot.append([0,0,0,1,0])
    elif train_label[i]==4:
        onehot.append([0,0,0,0,1])
#print(onehot_encoded)
# invert encoding

train_y=np.array(onehot)
print (train_y[1])

max stroke number in an image = 52 
 max point number in a stroke = 317
image number = 500
[[ 90.  62.]
 [ 92.  88.]
 [101. 139.]
 ...
 [  0.   0.]
 [  0.   0.]
 [  0.   0.]]
[0 0 0 1 0]


In [11]:
print (train_y.shape)
print (train_data.shape)

(500, 5)
(500, 16484, 2)


In [6]:
bs = 128 # batch size, each batch has n images
seq_len = 52*317 #seq_len = point_no * stroke_no, number of feature points in each image
input_size = 2 # number of features,  point
hidden_size = 32
output_size = 5 # n calsses
EPOCH = 2 # train the training data n times

In [7]:
model = Sequential()
model.add(LSTM(hidden_size, activation='tanh', return_sequences=False, input_shape=(seq_len,input_size)))
model.add(Dense(output_size, activation='linear', kernel_initializer='normal'))
model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                4480      
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 165       
Total params: 4,645
Trainable params: 4,645
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(
    optimizer=tf.train.AdamOptimizer(learning_rate=1e-3, ),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['accuracy']
)


model.fit(train_data, train_y,
          batch_size=bs,
          epochs=EPOCH,
          validation_split=0.2) # validation_data=(test_data, test_label)
#score, acc = tpu_model.evaluate(test_data, test_label,
#                            batch_size=batch_size)
#print('Test score:', score)
#print('Test accuracy:', acc)

Train on 400 samples, validate on 100 samples
Epoch 1/2


InvalidArgumentError: logits and labels must have the same first dimension, got logits shape [128,5] and labels shape [640]
	 [[Node: loss_1/dense_1_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits = SparseSoftmaxCrossEntropyWithLogits[T=DT_FLOAT, Tlabels=DT_INT64, _device="/job:localhost/replica:0/task:0/device:CPU:0"](loss_1/dense_1_loss/Log, loss_1/dense_1_loss/Cast)]]